In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os # Provide the functions that interact with the system,such as os.walk

cwd = os.getcwd() # Get the path to the current working directory


"""
# Determine if the working directory path exists
if os.path.exists(cwd):
    print('Directory exists')
else:
    print('Directory does not exist')
"""


# Input data files are available in the "/data/input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

print('filepath: ')
for dirname, _, filenames in os.walk(cwd + '\data\input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

print('relpath: ')
for dirname, _, filenames in os.walk(cwd + '\data\input'):
    for filename in filenames:
        filepath = os.path.join(dirname, filename)
        relpath = os.path.relpath(filepath)
        print(relpath)

# Any results you write to the current directory are saved as output.

filepath: 
D:\program\Kaggle\titanic\data\input\gender_submission.csv
D:\program\Kaggle\titanic\data\input\test.csv
D:\program\Kaggle\titanic\data\input\train.csv
relpath: 
data\input\gender_submission.csv
data\input\test.csv
data\input\train.csv


In [2]:
train_data = pd.read_csv("data/input/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("data/input/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
# train_data.loc() is a label-based data selection method for pandas dataframe. It allows you to select rows and columns by labels or Boolean array.
# Using train_data.loc() select a row with sex column 'female', and then select the 'survived' column.

women = train_data.loc[train_data.Sex == 'female']["Survived"]

# Data were filtered by filtering the rows with sex columns as 'female', and survival was then calculated by summing over the survived columns
# and dividing by the total number of females.

rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [5]:
# Survival rate was calculated for men.

men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


We'll build what's known as a random forest model. This model is constructed of several "trees" (there are three trees in the picture below, but we'll construct 100!) that will individually consider each passenger's data and vote on whether the individual survived. Then, the random forest model makes a democratic decision: the outcome with the most votes wins!

<img src="picture/random forest model.png" width="1100" height="600">

First the `RandomForestClassifier` class was imported from the `sklearn.ensemble` module, then the target variable `y` and the feature variable `features` were defined.

In [6]:
from sklearn.ensemble import RandomForestClassifier # 首先从sklearn.ensemble模块中导入了RandomForestClassifier类

# 定义了特征变量features和目标变量y
features = ["Pclass", "Sex", "SibSp", "Parch"]
y = train_data["Survived"]

"""
独热编码（One-Hot Encoding）是一种常用的对离散特征进行编码的方法。它将离散特征的每个类别都转换为一个二进制特征，其中1表示该样本属于该类别，0表示不属于。

举个例子，假设我们有一个名为color的离散特征，它有三个可能的取值：red、green和blue。我们可以使用独热编码将这个特征转换为三个二进制特征：is_red、is_green和is_blue。例如，如果一个样本的颜色为红色，那么它的独热编码后的特征值为is_red=1、is_green=0和is_blue=0。

独热编码通常用于将离散特征转换为适合机器学习模型使用的数值特征。它可以帮助模型更好地理解离散特征中不同类别之间的关系。
"""

# 使用pd.get_dummies()函数对训练数据和测试数据进行了独热编码
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])


`pd.get_dummies()`是`pandas`库中的一个函数，用于将分类变量转换为独热编码。

它接受一个`pandas`数据帧或系列对象，并根据分类变量中的唯一值创建新的二进制列。对于分类变量中的每个唯一值，它创建一个新的列，并将该值的出现标记为1，其余标记为0。
例如，如果有一个名为`"Sex"`的列包含两个唯一值`"male"`和`"female"`，则该函数将创建两个新的列`"Sex_female"`和`"Sex_male"`，并根据原始列中的值在相应的行中标记为1或0。

In [17]:
train_data[features]

,Pclass,Sex,SibSp,Parch
0,3,male,1,0
1,1,female,1,0
2,3,female,0,0
3,1,female,1,0
4,3,male,0,0
...,...,...,...,...
886,2,male,0,0
887,1,female,0,0
888,3,female,1,2
889,1,male,0,0


In [15]:
print(X)

     Pclass  SibSp  Parch  Sex_female  Sex_male
0         3      1      0           0         1
1         1      1      0           1         0
2         3      0      0           1         0
3         1      1      0           1         0
4         3      0      0           0         1
..      ...    ...    ...         ...       ...
886       2      0      0           0         1
887       1      0      0           1         0
888       3      1      2           1         0
889       1      0      0           0         1
890       3      0      0           0         1

[891 rows x 5 columns]


在上面的代码中，`pd.get_dummies()`被用于将训练数据和测试数据中的分类变量`"Pclass"`、`"Sex"`、`"SibSp"`和`"Parch"`转换为独热编码，以便它们可以被随后的随机森林分类器使用。

In [18]:
# 使用RandomForestClassifier函数建立一个随机森林模型，并使用fit函数将模型拟合到训练数据中
# n_estimators: 指定了使用的决策树的数量，
# max_depth: 指定了每棵决策树的最大深度，
# random_state: 指定了随机数生成器的种子，用于确保每次运行时结果一致
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)

# 使用fit函数将模型拟合到训练数据中
# 预测结果将存储在 predictions 变量中

model.fit(X, y)
predictions = model.predict(X_test)

In [21]:
# 使用 pd.DataFrame 函数创建一个新的 DataFrame，其中包含两列：PassengerId 和 Survived
# PassengerId 列的值来自 test_data.PassengerId，Survived 列的值来自 predictions 变量
# 使用 output.to_csv('data/output/submission.csv', index=False) 将 DataFrame 保存为 CSV 文件
    # index= 是 to_csv 函数的一个参数，False 表示在保存 CSV 文件时不写入行索引

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('data/output/submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
